https://github.com/uber/h3-py

https://deck.gl/examples/global-grids


```
conda config --add channels conda-forge
conda install h3-py
```

In [ ]:
import h3
import csv
from tqdm import tqdm

#
resolution = 6

shipTypes = ["Cargo", "Passenger", "Tanker"]

#dataPath = "/home/joajohan/Downloads/ais_small.csv"
dataPath = "/home/joajohan/Downloads/aisdk-2026-01-17.csv"

In [ ]:
def processFile(path, shipTypes, resolution):
    grids = {t: {} for t in shipTypes}

    # First count the lines to provide
    # a helpful (tqdm) progress bar
    lineCount = None
    with open(path) as f:
        lineCount = sum(1 for _ in f)

    with open(path) as csvfile:
        reader = csv.DictReader(csvfile)
        for r in tqdm(reader, total=lineCount):
            # Ignore other ship types
            if (r['Ship type'] not in shipTypes):
                continue
            # Bin position into cell
            cellId = h3.latlng_to_cell(
                float(r['Latitude']),
                float(r['Longitude']),
                resolution
            )
            # Add to count
            if (cellId not in grids[r['Ship type']]):
                grids[r['Ship type']][cellId] = 0
            grids[r['Ship type']][cellId] += 1
    return grids

def writeOutput(path, grids, shipType):
    with open(path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['id', 'value'])
        for i in grids[shipType].items():
            writer.writerow(i)

In [ ]:
grids = processFile(dataPath, shipTypes, resolution)

In [ ]:
for s in shipTypes:
    writeOutput(
        "../map/density{}.csv".format(s),
        grids,
        s
    )